In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='~/data/titanic/'

In [4]:
dfTrain = pd.read_csv('~/data/titanic/train.csv')
dfTest = pd.read_csv('~/data/titanic/test.csv')
dfFull = pd.concat([dfTrain, dfTest], ignore_index=True)


/home/awang/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
# Quickly fillna.
dfFull = dfFull.fillna(value={'Age':25, 'Sex':'male', 'Fare':1.0})

In [8]:
for col in ['Age', 'Sex', 'Fare']:
    cntMissing = len(dfFull[pd.isnull(dfFull[col])])
    if cntMissing > 0:
        print('%d passengers don\'t have %s specified' % (cntMissing, col))

In [9]:
cat_vars = ['Sex']
contin_vars = ['Age']
dep = 'Survived'

joined=dfFull[cat_vars + contin_vars + [dep]].copy()

# Change category columns to be ordered categorical
for v in cat_vars + [dep]: 
    joined[v] = joined[v].astype('category').cat.as_ordered()

# Change continuous columns to be float32
for v in contin_vars:
    joined[v] = joined[v].astype('float32')

In [14]:
n = 891
train_ratio = 0.9
val_idx = list(range(int(n*train_ratio), n))

# Split train+validation and test to meet fast.ai API
joined_tv = joined.loc[:n-1].copy()
joined_test = joined.loc[n:].copy()


df, y, nas, mapper = proc_df(joined_tv, dep, do_scale=True)
df_test, _, nas, mapper = proc_df(joined_test, dep, do_scale=True, 
                                  mapper=mapper, na_dict=nas)

In [15]:
nas

{}

In [16]:
cat_sz = [(c, len(joined_tv[c].cat.categories)+1) for c in cat_vars]
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [17]:
# Regression
# md = ColumnarModelData.from_data_frame('./', val_idx, df, y.astype(np.float32), cat_flds=cat_vars, bs=64, 
#                                        #is_reg=False, is_multi=False,
#                                        test_df=df_test)

# Classification
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, y.astype('int'), cat_flds=cat_vars, bs=4,
                                      is_reg=False,is_multi=False,test_df=df_test)

m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [10,5], [0.001,0.01],y_range=None, use_bn=True)

In [18]:
m.lr_find()
m.sched.plot(100)

  0%|          | 0/201 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generated/../generic/THCTensorMathReduce.cu:18